# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [24]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [8]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'
print("done")

done


In [10]:
# TODO: Define your transforms for the training, validation, and testing sets
data_transforms = transforms.Compose([transforms.RandomRotation(30),
                                     transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], 
                                                        [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], 
                                                        [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
image_datasets = datasets.ImageFolder(train_dir, transform = data_transforms)
test_image_datasets = datasets.ImageFolder(test_dir, transform = test_transforms)
validation_image_datasets = datasets.ImageFolder(valid_dir, transform = test_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = torch.utils.data.DataLoader(image_datasets, batch_size=64,shuffle=True)
testloaders = torch.utils.data.DataLoader(test_image_datasets, batch_size=64,drop_last=True)
validloaders = torch.utils.data.DataLoader(validation_image_datasets, batch_size=64,drop_last=True)
print("done")

done


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [18]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

print(cat_to_name)

{'21': 'fire lily', '3': 'canterbury bells', '45': 'bolero deep blue', '1': 'pink primrose', '34': 'mexican aster', '27': 'prince of wales feathers', '7': 'moon orchid', '16': 'globe-flower', '25': 'grape hyacinth', '26': 'corn poppy', '79': 'toad lily', '39': 'siam tulip', '24': 'red ginger', '67': 'spring crocus', '35': 'alpine sea holly', '32': 'garden phlox', '10': 'globe thistle', '6': 'tiger lily', '93': 'ball moss', '33': 'love in the mist', '9': 'monkshood', '102': 'blackberry lily', '14': 'spear thistle', '19': 'balloon flower', '100': 'blanket flower', '13': 'king protea', '49': 'oxeye daisy', '15': 'yellow iris', '61': 'cautleya spicata', '31': 'carnation', '64': 'silverbush', '68': 'bearded iris', '63': 'black-eyed susan', '69': 'windflower', '62': 'japanese anemone', '20': 'giant white arum lily', '38': 'great masterwort', '4': 'sweet pea', '86': 'tree mallow', '101': 'trumpet creeper', '42': 'daffodil', '22': 'pincushion flower', '2': 'hard-leaved pocket orchid', '54': 's

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [26]:
# TODO: Build and train your network
# use VGG networks
model = models.vgg16(pretrained=True)
model
# define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# move the model to use GPU if it is enabled
model.to(device)

for param in model.parameters():
    param.requires_grad = False
    
from collections import OrderedDict
# need to train the classifer for our data
# classifier = nn.Sequential(OrderedDict(
# [ ('fc1', nn.Linear(25088, 4096)),
#   ('relu', nn.ReLU()),
#   ('dropout', nn.Dropout(0.5)), 
#   ('fc2', nn.Linear(4096, 1000)),
#   ('relu', nn.ReLU()),
#   ('dropout', nn.Dropout(0.5)), 
#   ('fc3', nn.Linear(1000,102)),
#   ('output', nn.LogSoftmax(dim=1))
  
# ]))

hidden_units = 256
classifier = nn.Sequential(OrderedDict(
            [('fc1', nn.Linear(25088, hidden_units)),
                ('relu', nn.ReLU()),
                ('dropout', nn.Dropout(0.1)),
                ('fc2', nn.Linear(hidden_units, 102)),
                ('output', nn.LogSoftmax(dim=1))
                ]))
model.classifier = classifier

# define the loss
criterion = nn.NLLLoss()

learning_rate = 0.001

# define the optomizer
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)

#  the features size is 224x224
#model.classifier[0] = nn.Linear(25088, 1024)
print("done!")

cpu
done!


In [28]:
# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
#model.to ('cuda')
epochs = 1
steps = 0
testloop = 40
for epoch in range(epochs):
    running_loss = 0
    for images, labels in dataloaders:
        steps += 1

        # move input, label tensores to the device
        images, labels = images.to(device), labels.to(device)
        # reset the gradient
        optimizer.zero_grad()
        # calculate the log probability
        # we should get 10 class probabilities for 64 examples
        logps = model.forward(images)

        # calculate the loss
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss +=loss.item()
        print("new loss...")
        print(loss.item())
        
        # in every 5 loops, we will use the test data to check the model's accuracy
        if steps % testloop  == 0:
            test_loss = 0
            accuracy = 0
            print(f"running_loss {running_loss}.. ")
            # set model to evaluation mode (non-training)
            print("switch to eval mode")
            model.eval()
            # no gradiation calculation for test data
            with torch.no_grad():
                for images, labels in testloaders:
                    images, labels = images.to(device), labels.to(device)
                    test_logps = model.forward(images)
                    loss = criterion(test_logps, labels)
                    test_loss += loss.item()
                    
                    # get the probability
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    # equals = top_class == labels.view_as(top_class)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
                    
                    print(f"Epoch {epoch+1}/{epochs}.. "
                          f"Train loss: {running_loss/testloop:.3f}.. "
                          f"Test loss: {test_loss/len(testloaders):.3f}.. "
                          f"Test accuracy%: {accuracy/len(testloaders)*100:.3f}")
            # set the modal back to training
            model.train()
            print("reset to train mode")
print("training completed!")            
        
        

cpu
new loss...
4.622006893157959
new loss...
9.322637557983398
new loss...
8.945881843566895
new loss...
9.70717716217041
new loss...
9.21966552734375
new loss...
6.078012943267822
new loss...
5.77942419052124
new loss...
5.0223259925842285
new loss...
4.443029880523682
new loss...
4.35037899017334
new loss...
4.252099990844727
new loss...
3.97281813621521
new loss...
3.8975017070770264
new loss...
3.956355333328247
new loss...
4.00963020324707
new loss...
3.487180471420288
new loss...
3.8342926502227783
new loss...
3.8826000690460205
new loss...
3.3705291748046875
new loss...
3.4557838439941406
new loss...
3.686938524246216
new loss...
3.5016238689422607
new loss...
3.4024758338928223
new loss...
3.099820375442505
new loss...
3.1848092079162598
new loss...
3.2066173553466797
new loss...
3.4791531562805176
new loss...
3.3143692016601562
new loss...
3.1966640949249268
new loss...
3.1059961318969727
new loss...
3.128103017807007
new loss...
2.914905309677124
new loss...
2.86629676818847

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
def validation(model, validloader, criterion):
    accuracy = 0
    test_loss = 0
    with torch.no_grad():
        print("start validation")
        for images, labels in validloader:

            output = model.forward(images)

            ## Calculating the accuracy 
            # Model's output is log-softmax, take exponential to get the probabilities
            ps = torch.exp(output)
            # Class with highest probability is our predicted class, compare with true label
            #equality = (labels == ps.max(1)[1])
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            # Accuracy is number of correct predictions divided by all predictions, just take the mean
            # accuracy += equality.type_as(torch.FloatTensor()).mean()
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

    return accuracy


In [35]:

accuracy = validation(model, validloaders, criterion)
print(f"what is the accuracy? {accuracy:.3f}")

start validation
what is the accuracy? 6.547


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# save the checkpoint 
model.class_to_idx = image_datasets['train'].class_to_idx
checkpoint = {
    'epoch': epoch, 
    'model_state_dict': model.state_dict(),  
    'optimizer_state_dict': optimizer.state_dict(),
    'classifier':classifier,
    'class_to_idx':model.class_to_idx
}

torch.save(checkpoint, 'save_model/checkpoint2.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint_with_file(checkpointfile):
    checkpoint = torch.load('checkpoint.pth')  
    checkpoint = torch.load(checkpointfile)
    model = models.vgg11(pretrained=True)
    model.load_state_dict (checkpoint['model_state_dict'])
    model.classifier = checkpoint['classifier']
    model.class_to_index = checkpoint['class_to_idx']

    for param in model.parameters():
        param.requires_grad = False

    return model
    
filepath = 'save_model/checkpoint2.pth'
loaded_model = load_checkpoint_with_file(filepath) 

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # TODO: Process a PIL image for use in a PyTorch model
    with Image.open("hopper.jpg") as im:
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224)
            transforms.ToTensor(),                  
            transforms.Normalize(                   
                mean=[0.485, 0.456, 0.406],         
                std=[0.229, 0.224, 0.225]
            )
        ])
        pil_image = preprocess(im)
        np_image = np.array(pil_image)

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
image_path = 'flowers/train/1/image_06734.jpg'
img = process_image(image_path)
print(img)
imshow(img)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    # eg. image_path = 'flowers/train/1/image_06734.jpg'
    img = process_image(image_path)
    img = img.unsqueeze(0)
    # TODO: Implement the code to predict the class from an image file
    model.eval()
    with torch.no_grad():
        output = model(image)
    # from logits to probabilities    
    ps = torch.softmax(output, dim=1)
    
    # get the topK probablities and index
    #[batchsize, top k of probability], [batchsize, top k of index]
    top_pros, top_indices = ps.topk(topk)   
    # convert from these indices to the actual class labels 
    # using class_to_idx
    top_pros = top_pros.numpy().squeeze() #  [0.3, 0.4, 0.1, 0.1, 0.1] 
    top_indices = top_indices.numpy().squeeze()
    # convert indices to class
    idx_to_class_map = {idx: cls for cls, idx in model.class_to_index.items()}
    # get the top 5 class
    top_class = [idx_to_class_map[idx] for idx in top_indices]
    return top_pros, top_class

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
import matplotlib.pyplot as plt
def show_prob_with_image(image_path, top5_ps, top5_class):
    img = process_image(image_path)
        
    fig, (ax1,ax2) = plt.subplots(figsize=(6,9), nrows=2)
    img = img.numpy().transpose((1, 2, 0))
    print(f"after transpose {img.shape}") 

    ax1.imshow(np.clip(img, 0, 1))
    ax1.axis('off')
    ax2.barh(np.arange(5), top5_ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(5))
    flower_names = [cat_to_name.get(cls, "Unknown") for cls in top5_class]
    # debug
    print("can we get the flower names?")
    print(flower_names)
    ax2.set_yticklabels(flower_names)
    # TOOD: title need to set the max probabilty flower name!! 
    print("what is max ps index?")
    print(top5_ps.index(np.max(top5_ps).item()))
    top_index = top5_ps.index(np.max(top5_ps).item())
    title = cat_to_name[str(top5_class[top_index])]
    print(title)
    ax1.set_title(title)
    ax2.set_xlim(0,1.1)
    
    plt.tight_layout()
    plt.show()

In [ ]:
def show_prob_with_image_topk(img, top_ps, top_class, top_k=5):
    ''' Display an image along with the top K classes
    '''
    fig, (ax1,ax2) = plt.subplots(figsize=(6,9), nrows=2)
    img = img.numpy().transpose((1, 2, 0))
    print(f"after transpose {img.shape}") 

    ax1.imshow(np.clip(img, 0, 1))
    ax1.axis('off')
    ax2.barh(np.arange(top_k), top_ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(top_k))
    flower_names = [cat_to_name.get(cls, "Unknown") for cls in top_class]
    # debug
    print("can we get the flower names?")
    print(flower_names)
    ax2.set_yticklabels(flower_names)
    # title needs to set the max probabilty flower name.
    top_ps = top_ps.tolist()
    top_index = top_ps.index(max(top_ps))
    top_index = top_ps.index(np.max(top_ps).item())
    title = cat_to_name[str(top_class[top_index])]
    print(title)
    ax1.set_title(title)
    ax2.set_xlim(0,1.1)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# predict the image and show the top 5 class with highest probability results
image_path = 'flowers/train/1/image_06734.jpg'
top5_ps, top5_class = = predict(image_path, loaded_model, topk=5)
show_prob_with_image(process_image(image_path),
                             top5_ps, top5_class, topk)